![imagenes](logo.png)

# Funciones de activación

Una neurona (antes de activación) calcula $z=\boldsymbol{w}\cdot\boldsymbol{a}+b$. Ahora bien, la función $F(\boldsymbol{t})=\boldsymbol{w}\cdot\boldsymbol{t}+b$ es una función afín. Si tienes muchas capas, y no usas funciones de activaciones, el flujo de la información se ve así:

$$\boldsymbol{x}\stackrel{\text{afín}}{\longrightarrow}\boldsymbol{z}_1\stackrel{\text{af\'in}}{\longrightarrow}\boldsymbol{z}_2...$$

Pero, matemáticamente, la composición de funciones afines vuelve a ser afín. Por lo tanto, aunque tengas 100 capas, el resultado final sigue siendo lineal. 

## Definición de función de activación

La **función de activación en la capa $i$** es una función $f_i:\mathbb{R}\to\mathbb{R}$ que se aplica al pensamiento crudo $z_i^{(j)}$ de cada neurona $j$ de la capa, y cuyo propósito es **romper la linealidad del modelo**. No agrega información nueva ni introduce memoria, sino que introduce **no linealidad estructural**. 

La activación decide si la neurona se expresa, cuánto se expresa y en qué rango vive su salida.

Para que una activación sea útil ha de cumplir:
- No ser lineal
- Ser derivable en casi todas partes
- No anular derivadas sistemáticamente
- Ser computacionalmente barata

No todas cumplen todo perfectamente. Por esa razón existen varias.

## Ejemplos de activaciones

### Activaciones duras (históricas)

La función escalón $f(x)=\left\{\begin{array}{l}1\mbox{ si }x\ge0\\0\mbox{ si }x<0\end{array}\right.$

- Toma una decisión binaria
- No es derivable en 0
- No es entrenable por propagación hacia atrás.

Fue la primera en ser usada históricamente, pero hoy en día es inútil.

### Activaciones suaves (saturantes)

La sigmoide $f(x)=\frac{1}{1+e^{-x}}$

- Su rango es $(0,1)$
- Por lo tanto tiene una interpretación probabilística
- Su derivada es casi 0 cuando $x$ es grande

El problema es que sus gradientes se desvanecen: la sigmoide apaga el gradiente porque, cuando la neurona está muy convencida, deja de aprender.

La tangente hiperbólica $f(x)=\tanh(x)$

- Tiene rango en (-1,1)
- Está centrada en 0
- Es mejor que la sigmoide, pero presenta el mismo problema.

### Activaciones no saturantes

La ReLU (unidad lineal rectificante) dada por $f(x)=\max(0,x)$

- Es lineal para $x>0$;
- Es $0$ para $x\le 0$.
- Tiene una derivada sencilla.

Esta función cambió al Deep Learning.

### La Geometría de ReLU

Por simplicidad, supongamos que la capa $i$ tiene solo $n_i = 2$ neuronas y la capa $i + 1$ tiene $n_{i+1} \geq 2$ neuronas. Cada neurona de la capa $i + 1$ tiene entonces dos pesos y un sesgo.

Fijemos la neurona $j$ de la capa $i + 1$. Antes de recibir información de la capa anterior, esta neurona tiene a priori asociado un plano en $\mathbb{R}^3$, dado por

$$
F_j(x, y) = w_{i+1,1}^{(j)} x + w_{i+1,2}^{(j)} y + b_{i+1}^{(j)}.
$$

El punto $(x, y, F_j(x, y))$ pertenece siempre a este plano. En particular, al evaluar en la activación de la capa anterior $a_i = (a_i^{(1)}, a_i^{(2)})$, se obtiene

$$
F_j(a_i^{(1)}, a_i^{(2)}) = z_{i+1}^{(j)}.
$$

Cada uno de estos planos intersecta al plano horizontal $z = 0$ en una recta, y el conjunto de estas rectas particiona el plano $(x, y)$ en varias regiones, formando una teselación geométrica.

Cuando la capa $i + 1$ recibe la activación $a_i$, la función ReLU de la neurona $j$ evalúa el valor $z_{i+1}^{(j)}$. Si $z_{i+1}^{(j)} \leq 0$, el punto $(a_i^{(1)}, a_i^{(2)}, z_{i+1}^{(j)})$ queda en o por debajo del plano $z = 0$, la neurona se apaga y su salida es cero, por lo que no transmite información a la capa $i + 2$. Si $z_{i+1}^{(j)} > 0$, el punto queda por encima del plano $z = 0$ y la neurona transmite a la capa siguiente su valor lineal $z_{i+1}^{(j)}$.

De este modo, la región de la teselación en la que cae $a_i$ determina qué neuronas de la capa $i + 1$ se activan y cuáles se apagan, y esta información queda codificada en el vector que recibe la capa $i + 2$.

## Función de activación de la capa de salida

Hasta ahora hemos entendido a la red neuronal como un proceso de transformación: entra un cliente con características → pasa por filtros (capas ocultas) → y al final la red debe tomar una decisión.

Esa decisión final se concreta exclusivamente en la capa de salida, y la función de activación que se elige ahí no es estética ni arbitraria:
está dictada por la naturaleza del problema.

En las capas ocultas, la activación tiene el rol de introducir no linealidad, permitir representaciones complejas y facilitar el aprendizaje. En la capa de salida, la activación tiene otro rol: **traducir números internos de la red a algo que tenga significado en el mundo real.**

Por ejemplo, una probabilidad, clase, puntaje, cantidad, decisión binaria. Por eso no todas las activaciones sirven para todas las tareas.

### Tipos de problemas activación adecuada

#### Problemas de regresión

La capa de salida tiene una sola neurona. La salida puede ser cualquier número real y se busca predecir dicho número. Por ejemplo: el precio de una casa, la temperatura de un lugar, las ventas en $ de una empresa, coordenadas donde impactará un proyectil. 

Aquí se suele tomar $f_{out}(t)=t$.

#### Clasificación binaria (sí/no)

La capa de salida tiene una sola neurona. La salida suele ser una probabilidad: $P(y=1|\boldsymbol{x})$. Por ejemplo: un correo es spam o no; un cliente entra o no; un paciente tiene cierta enfermedad o no. 

Se suele tomar $f_{out}(t)=\frac{1}{1+e^{-t}}$

#### Clasificación multiclase (una sola clase correcta)

La capa de salida tiene varias neuronas. En este caso queremos varias salidas: la probabilidad de pertenecer a cada clase, siendo solo una de estas clases la correcta. Por ejemplo, la imagen de un dígito del 0 al 9; un animal en una imagen; un tipo de flor.

Se suele tomar $$f_{out}(z_{out}^{(j)})=\mathrm{softmax}(z_{out}^{(j)})=\frac{e^{z_{out}^{(j)}}}{\sum e^{z_{out}^{(k)}}}$$

Cada neurona es una sola clase. Las salidas son las probabilidades de cada clase. Varias puertas compiten pero solo una se abre.

#### Clasificación multietiqueta (varias respuestas posibles)

La capa de salida tiene varias neuronas.

Una imagen puede contener a un perro y a un gato; o un correo puede ser spam y promoción.

En este caso $f_{out}(z_{out}^{(j)})=\frac{1}{1+e^{-z_{out}^{(j)}}}$

Cada salida es independiente; no hay competencia y cada neurona responde ¿está o no está?

